# Домашнее задание к лекции «Случайные события. Случайные величины»

Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб., после чего выпадает случайное целое число, равномерно распределенное от 0 до 999. При некоторых значениях числа игрок получает выигрыш.

777: 200 руб.
<br>999: 100 руб.
<br>555: 50 руб.
<br>333: 15 руб.
<br>111: 10 руб.
<br>*77: 5 руб.
<br>**7: 3 руб.
<br>*00: 2 руб.
<br>**0: 1 руб.

Вопросы:

Выгодна ли игра игроку?
Сколько в среднем приобретает или теряет игрок за одну игру?

**Дополнительное задание повышенной сложности.**

Теоретически рассчитать средний выигрыш (проигрыш) и сравнить с результатами моделирования

– любая цифра

## Решение 1. Эксперимент

In [421]:
import numpy as np

In [422]:
# преднастройка выигрышных слотов
win_slots = {
            '777': 200,
            '999': 100,
            '555': 50,
            '333': 15,
            '111': 10,
            '77': 5,
            '7': 3,
            '00': 2,
            '0': 1
}

Определять выигрышность конкретного случая будем через отдельную функцию

In [423]:
def win_check(attempt_value, win_slots):
    '''
    Возвращает сумму выигрыша
    
    attempt_value - сколько выпало
    win_slots - преднастроеный списко выигрышных комбинаций
    '''
    win_res = 0

    if attempt_value == 0:
        win_res = win_slots['00']
    
    elif str(attempt_value) in win_slots.keys():
        win_res = win_slots[str(attempt_value)]
            

    elif attempt_value % 100 == 77:
        win_res = win_slots['77']
    elif attempt_value % 10 == 7: 
        win_res = win_slots['7']

    elif (
            attempt_value % 100 == 0 and
            attempt_value % 10 == 0
    ):
        win_res = win_slots['00']
    elif attempt_value % 10 == 0: 
        win_res = win_slots['0']

    else:
        win_res == 0
        
    return win_res

# win_check(0, win_slots)
# # win_slots['77']

Тестирование определителя выигрыша

In [424]:
# используется для тестовых целей
test_slots = {
    '777': 200,
    '999': 100,
    '555': 50,
    '333': 15,
    '111': 10,
    '177': 5, '077': 5,
    '127': 3, '007': 3,
    '300': 2, '000': 2,
    '230': 1, '010': 1
}

In [425]:
# тест работы определителя
count = 1
for key, win in test_slots.items():
    print(count, key, win_check(int(key), win_slots))
    count += 1

print('Всего тестов:', len(test_slots))

1 777 200
2 999 100
3 555 50
4 333 15
5 111 10
6 177 5
7 077 5
8 127 3
9 007 3
10 300 2
11 000 2
12 230 1
13 010 1
Всего тестов: 13


*все нормально, тестовые примеры определились корректно, включая "000"*

### Постановка эксперимента

* Выгодна ли игра игроку? 
* Сколько в среднем приобретает или теряет игрок за одну игру?

In [426]:
# Количество игра - попыток
attempts_qty = 10**6

# счетчики - накопители
total_win = 0
total_spent = 0
clear_win = 0

one_bet = 1 # Игрок платит 1 руб.

# эксперимент
for attempt in range(1, attempts_qty + 1):

    attempt_res = np.random.randint(0, 1000)

    win_res = win_check(attempt_res, win_slots)
    
    if win_res > 0:
        total_win += win_res

    total_spent += one_bet 
    
    clear_win = total_win - total_spent
    avg_win_loss = clear_win / attempts_qty

    
print(total_win, total_spent, clear_win, avg_win_loss)
print('Средний проигрыш (-), выигрыш(+) составляет:', round(avg_win_loss, 4))

795173 1000000 -204827 -0.204827
Средний проигрыш (-), выигрыш(+) составляет: -0.2048


### Ответы
1) Игра не выгодная, в большинстве экспериментов на 1 млн попыток ИТОГО = минус (на 1 тыс попыток иногда попадается ПЛЮС)
2) Средний проигрыш за одну игру составляет 20 копеек

## Решение 2. Теоретический расчет

* Для решения необходимо рассчитать М - матожидание.
* Результат теоретического расчета должен быть близок в экспериментальному

M = сумма всех положительных и отрицательных исходов: вероятность * на их доход (выиграл - потратил) 

In [427]:
win_slots

{'777': 200,
 '999': 100,
 '555': 50,
 '333': 15,
 '111': 10,
 '77': 5,
 '7': 3,
 '00': 2,
 '0': 1}

**Расчет количества каждого исхода**

In [428]:
full_range = range(0, 1000) #0 - 999
len_full_range = len(full_range)
win_keys = list(win_slots.keys())

# Расчет количества выигрышей, подготовка для расчета вероятностей
q777_200 = 1
q999_100 = 1
q555_50 = 1
q333_15 = 1
q111_10 = 1

q77_5_count = 0
q7_3_count = 0
q00_2_count = 0
q0_1_count = 0

for i in full_range:

    if i == 0:
        q00_2_count += 1
    
    elif i % 100 == 77:
        q77_5_count += 1
    elif i % 10 == 7: 
        q7_3_count += 1

    elif (
            i % 100 == 0 and
            i % 10 == 0
    ):
        q00_2_count += 1
        
    elif i % 10 == 0: 
        q0_1_count += 1

# обработка комбинации 777 - надо уменьшить количество "*77" на одну комбинацию 
q77_5_count -= q777_200
        
qty_win_list = [
        q777_200,
        q999_100,
        q555_50,
        q333_15,
        q111_10,
        q77_5_count,
        q7_3_count,
        q00_2_count,
        q0_1_count
]

# И количество проигрышных комбинация
q_loss = len_full_range - sum(qty_win_list)

qty_win_dict = dict(zip(win_keys, qty_win_list))
print("Сколько раз встречается комбинация в Range(1000):")
display(qty_win_dict)
print('Всего выигрышных: {0}\nПроигрышных: {1}\nИз: {2}'
      .format(
          sum(qty_win_dict.values()), 
          q_loss, 
          sum(qty_win_dict.values()) + q_loss)
     )

Сколько раз встречается комбинация в Range(1000):


{'777': 1,
 '999': 1,
 '555': 1,
 '333': 1,
 '111': 1,
 '77': 9,
 '7': 90,
 '00': 10,
 '0': 90}

Всего выигрышных: 204
Проигрышных: 796
Из: 1000


**Расчет матожидания и проверка эксперимента**

In [429]:
M = 0
print('М для каждого шага')
print('Комб-я / Кол-во / из ВСЕГО / разовый выигрыш / вероятная сумма')
for key, el in qty_win_dict.items():
    print(key, el, len_full_range, win_slots[key], 'P=', (el / len_full_range) * win_slots[key])
    M += (el / len_full_range) * (win_slots[key] - one_bet)

M += (q_loss / len_full_range) * (-one_bet)

М для каждого шага
Комб-я / Кол-во / из ВСЕГО / разовый выигрыш / вероятная сумма
777 1 1000 200 P= 0.2
999 1 1000 100 P= 0.1
555 1 1000 50 P= 0.05
333 1 1000 15 P= 0.015
111 1 1000 10 P= 0.01
77 9 1000 5 P= 0.045
7 90 1000 3 P= 0.27
00 10 1000 2 P= 0.02
0 90 1000 1 P= 0.09


## 3. Сверка Эксперимента и Теории

In [435]:
print('\n')
print('Средний проигрыш (-), выигрыш(+) составляет:')
print(f'1 - Экспериментально ("сыграли" {attempts_qty} раз):', round(avg_win_loss, 4))
print('2 - Теоретический расчет (М = матожидание):', round(M, 4))



Средний проигрыш (-), выигрыш(+) составляет:
1 - Экспериментально ("сыграли" 1000000 раз): -0.2048
2 - Теоретический расчет (М = матожидание): -0.2


**Данные эксперимента подтверждены теоретическим расчетом**